In [89]:
# imports
import pandas as pd
import os

In [90]:
# read source data
dir = os.getcwd()
df = pd.read_csv(dir + r'\data\recipes_w_search_terms.csv')

# create datasets
<br> df_classic: all recipes [all]
<br> df_classic_reduced: all recipes that are not vegetarian (and therefore also not vegan) [not vegetarian, not vegan]
<br> df_vegetarian: all recipes that are vegetarian [vegetarian]
<br> df_vegetarian_reduced: all recipes that are vegetarian but not vegan [vegetarian, not vegan]
<br> df_vegan: all recipes that are vegan [vegan]

In [91]:
# # create classic datasets
# df_classic = df[['id', 'name', 'ingredients', 'search_terms']]
# df_classic_reduced = df_classic[~df_classic['search_terms'].str.contains('vegetarian')]
# df_classic.to_csv(dir + r'\data\df_classic.csv')
# df_classic_reduced.to_csv(dir + r'\data\df_classic_reduced.csv')

In [92]:
# # create vegetarian datasets
# df_vegetarian = df_classic[df_classic['search_terms'].str.contains('vegetarian')]
# df_vegetarian_reduced = df_vegetarian[~df_vegetarian['search_terms'].str.contains('vegan')]
# df_vegetarian.to_csv(dir + r'\data\df_vegetarian.csv')
# df_vegetarian_reduced.to_csv(dir + r'\data\df_vegetarian_reduced.csv')

In [93]:
# # create vegan datasets (df_vegan == df_vegan_reduced)
# df_vegan = df_classic[df_classic['search_terms'].str.contains('vegan')]
# df_vegan.to_csv(dir + r'\data\df_vegan.csv')

# data cleaning

In [99]:
# drop recipes with no name
droplist = ['---', '------------', '--------------', '----------------', '-------------------', '----------------------']
for element in droplist:
    df.drop(df[df.name == element].index, inplace=True)

In [114]:
df.sort_values('name')[650:700]

,id,name,description,ingredients,ingredients_raw_str,serving_size,servings,steps,tags,search_terms,label
194772,418039,&quot;crab&quot; Dip,This is a very good cold dip. You can use real...,"['surimi', 'sour cream', 'cream cheese', 'mayo...","[""4 ounces surimi (Imitation Crab)"",""4 ...",1 (44 g),10,"['Shred surimi.', 'Beat cream cheese,sour crea...","['15-minutes-or-less', 'time-to-make', 'course...","{'appetizer', 'low-carb'}",classic
326690,317696,&quot;crab&quot; Stuffed Mushrooms,the imitation crab helps the cholesterol count,"['baby portabella mushrooms', 'imitation crabm...","[""12 ounces fresh baby portabella mushroom...",1 (599 g),1,"['Combine crab in a medium bowl with onions, o...","['60-minutes-or-less', 'time-to-make', 'course...","{'diabetic', 'dinner'}",classic
69631,168644,&quot;crabby&quot; Bread!,"This is SO CUTE!!! Place ""crab"" onto lettuce-l...","['rhodes frozen rolls', 'toothpick', 'butter',...","[""5 Rhodes frozen rolls, texas, thawed b...",1 (20 g),4,"['For the body, join 2 rolls together and pres...","['60-minutes-or-less', 'time-to-make', 'course...","{'appetizer', 'bread'}",classic
399430,243157,&quot;crack&quot; Lasagna,This is a favorite of my family and friends. ...,"['sweet italian sausage', 'broccoli rabe', 'ol...","[""1 lb sweet Italian sausage (I like turk...",1 (234 g),9,"['Brown and crumble sausage in a skillet.', 'W...","['time-to-make', 'course', 'main-ingredient', ...",{'dinner'},classic
23558,160170,&quot;crazy Easy&quot; Lo Mein,We don't pay $7 a plate for Lo Mein any more. ...,"['spaghetti noodles', 'cooked ground beef', 'b...","[""16 ounces spaghetti noodles, cooked and...",1 (271 g),4,"['Combine the cooked hamburger, both jars of b...","['30-minutes-or-less', 'time-to-make', 'course...",{'dinner'},classic
298451,271674,&quot;cream of ...&quot; Soup - Substitute,How to make your own Cream of whatever soup,"['butter', 'flour', 'salt', 'milk', 'chicken s...","[""2 tablespoons butter"",""2 tablespoons ...",1 (450 g),1,"['In a heavy saucepan, melt the butter. Sprink...","['15-minutes-or-less', 'time-to-make', 'course...","{'low-calorie', 'soup', 'low-sodium', 'low-carb'}",classic
425538,441744,&quot;cream of Whatever&quot; Soup,"A healthy, budget-friendly substitute for cann...","['butter', 'flour', 'garlic clove', 'milk', 'o...","[""2 tablespoons butter"",""2 tablespoons ...",1 (48 g),4,"['In a small saucepan over medium heat, melt b...","['weeknight', '15-minutes-or-less', 'time-to-m...","{'diabetic', 'low-calorie', 'low-sodium', 'low...",classic
492574,271113,&quot;cream&quot; of Chicken Soup,"From ""Dining In""; I've always wanted to find a...","['onion', 'oil', 'water', 'raw rice', 'chicken...","[""1 onion, diced "",""1 tablespoon oi...",1 (253 g),6,"['Saute onion in oil until clear.', 'Add water...","['60-minutes-or-less', 'time-to-make', 'course...","{'low-calorie', 'low-sodium', 'healthy', 'low-...",classic
111971,380724,&quot;cream&quot; of Chicken and Vegetable Sou...,I won't even go into this....I made it and UGH...,"['chicken breasts', 'water', 'low sodium chick...","[""4 large chicken breasts (do not chop)"",...",1 (688 g),6,"['Add water, cumin, garlic &amp; bay leaves, b...","['time-to-make', 'course', 'main-ingredient', ...","{'low-calorie', 'soup', 'low-sodium', 'low-carb'}",classic
280122,284990,"&quot;creamed&quot; Rosemary Potatoes (Vegan, ...",Many people have asked me for this recipe. It...,"['olive oil', 'split red lentils', 'red potato...","[""1/2 cup olive oil"",""1 cup split red...",1 (874 g),6,"['Rinse lentils thoroughly. Drain.', ""Bring w...","['60-minutes-or-less', 'time-to-make', 'main-i...","{'gluten-free', 'vegetarian', 'vegan'}",vegan


# add labels 

In [94]:
# df = df[['id', 'name', 'ingredients', 'search_terms']]

In [95]:
df = df.assign(label=['vegan' if 'vegan' in x else 'vegetarian' if 'vegetarian' in x else 'classic' for x in df.search_terms])

In [98]:
df[['id', 'name', 'ingredients', 'label']].to_csv(dir + r'\data\df_preprocessed.csv', index = False)